# Understanding missing muscles for QUMIA project

In [ ]:
## import libraries
import os       # using operating system dependent functionality (folders)
import glob
import shutil 
import itertools
import tempfile 
import subprocess

import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets

from sklearn import metrics
from sklearn.metrics import confusion_matrix

import datetime
from shutil import copytree

from IPython.display import Image

import imageio
import skimage

from IPython.display import Image

In [ ]:
## import data
data= pd.read_csv('merged.csv')

In [ ]:
standard = [
    'm. deltoideus',
    'm. biceps brachi',
    'm. flexor digitorum profundus',
    'm. interosseus dorsalis',
    'm. rectus femoris',
    'm. vastus lateralis',
    'm. tibialis anterior',
    'm. gastrocnemius mediale kop',
] 

myositis = [
    'm. deltoideus',
    'm. biceps brachi',
    'm. interosseus dorsalis',
    'm. rectus femoris',
    'm. tibialis anterior',
    'm. gastrocnemius mediale kop',
    'm. flexor digitorum profundus',
    'm. vastus lateralis',
    'm. gastrocnemius mediale kop',
]
        
    
mnd = [
    'm. geniohyoideus (unilateraal)',
    'm. digastricus',
    'm. masseter',
    'm. sternocleidomastoideus',
    'm. trapezius',
    'm. deltoideus',
    'm. biceps brachi',
    'm. rectus femoris',
    'm. flexor digitorum profundum',
    'm. vastus lateralis',
    'm. gastrocnemius mediale kop',
     'm. tibialis anterior',
     'm. tibialis anterior',
]
    
    
fshd = [
    'm. trapezius',
    'm. deltoideus',
    'm. biceps brachi',
    'm. rectus abdominalis',
    'm. vastus lateralis',
    'm. rectus femoris',
    'm. tibialis anterior',
    'm. gastrocnemius mediale kop',
    'm. flexor digitorum profundum',
    'm. vastus lateralis',
]
 

In [ ]:
list_of_sets = [standard,  myositis, mnd, fshd]

In [ ]:
set_list = []
for set_e in list_of_sets:
    set_e = set(set_e)
    set_list.append(set_e)
    
set_list[1]    

In [ ]:
set_list[2]

In [ ]:
set_list[3].intersection(set_list[0].intersection(set_list[1].intersection(set_list[2])))

In [ ]:
print(set_list[0] - set_list[1])

In [ ]:
# optinal examine
data.head(10)

In [ ]:
data.columns

In [ ]:
reduced_data = data[['exam_id','muscle', 'diagnosis']]

In [ ]:
reduced_data

In [ ]:
further_reduced_data = reduced_data.drop_duplicates()

In [ ]:
further_reduced_data

In [ ]:
def categorize(x):
    
    return set(x)
result = further_reduced_data.groupby(['exam_id'])[['muscle', 'diagnosis']].agg([categorize])
result

In [ ]:
result.columns

In [ ]:
result.columns = result.columns.to_flat_index()

In [ ]:
result = result.reset_index()
result

In [ ]:
result.columns

In [ ]:
result = result.rename(columns={('exam_id', ''): 'exam_id', ('muscle', 'categorize'): 'muscle_set',  ('diagnosis', 'categorize'): 'diagnosis'})
result

In [ ]:
result['diagnosis'] = result['diagnosis'].apply(list)
result['diagnosis'] = result['diagnosis'].apply(lambda x:x[0])

In [ ]:
result

In [ ]:
result.diagnosis.unique()

In [ ]:
result['muscle_set'] = result['muscle_set'].apply(list)

In [ ]:
result['muscle_set'] = result['muscle_set'].apply(sorted)

In [ ]:
result['muscle_set'] = result['muscle_set'].apply(lambda x :" ".join(x))

In [ ]:
# add a reorganize to make the muscle sets, the same

In [ ]:
result

In [ ]:
len(result.muscle_set.unique())

In [ ]:
result.muscle_set.value_counts()

In [ ]:
result.muscle_set.value_counts().sort_values().plot(kind = 'barh')

In [ ]:
result.muscle_set.value_counts().sort_values().tail(10).plot(kind = 'barh')

I will go looking to see if there are any mis-spellings in the ones where we only have one value

In [ ]:
result.muscle_set.value_counts().sort_values().tail()

In [ ]:
result.muscle_set.value_counts().sort_values().head(20)

Everything looks regularly spelled, as if by a drop-down menu. CHeck with LA. 

Anyways..let's see if there is variety in the diagnoses by muscle set.

In [ ]:
top_set_5 = result[result['muscle_set'] == 'Biceps Deltoid FCR GM ID1 PerT TA VL']
top_set_5['diagnosis'].value_counts()

In [ ]:
top_set_4 = result[result['muscle_set'] == 'Biceps Deltoid Digastric GM Geniohyoid Masseter RF Rectusab TA Trap VL']
top_set_4['diagnosis'].value_counts()

## Uhoh, we have a problem ...whenever we have the fourth most popular set of muscles, the diagnosis is always 5

In [ ]:
top_set_3 = result[result['muscle_set'] == 'Biceps Digastric FCR GM Geniohyoid ID1 Masseter RF Rectusab SCM TA Trap']
top_set_3['diagnosis'].value_counts()

In [ ]:
top_set_2 = result[result['muscle_set'] == 'Biceps Deltoid FCR GM TA VL']
top_set_2['diagnosis'].value_counts()

In [ ]:
top_set_1 = result[result['muscle_set'] == 'Biceps Deltoid FCR GM ID1 PerT TA VL']
top_set_1['diagnosis'].value_counts()

## So most sets in the top 5 have a nice spread of diagnosis with this wierd excepetion of the fourth

## WE can also look the oher way, at diagnosis and see if they always map to a certain set of muscles.

In [ ]:
result[result['diagnosis'] == 5]

In [ ]:
result[result['diagnosis'] == 3]

In [ ]:
result[result['diagnosis'] == 0]

In [ ]:
result[result['diagnosis'] == 99]

In [ ]:
print(type(result['muscle_set'][0])) 